<a href="https://colab.research.google.com/github/Youngdeuk-Koo/KNLP/blob/main/TextRank%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EB%8F%99%ED%99%94_%EC%9A%94%EC%95%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git 
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh
%cd ../

#0. 시스템 재귀 횟수 오류 방지용

In [2]:
import sys

def func(n):
    # if not n%100:print(n)
    if n == 14900:
        return
    func(n + 1)

if __name__=='__main__':
    sys.setrecursionlimit(15000)
    func(1)

In [3]:
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

#1. 전처리된 텍스트 데이터 불러오기(아기 돼지 삼형제)

In [4]:
with open('/content/drive/MyDrive/AI_Study/NLP/DATA/소설 및 동화/아기돼지3형제.txt', 'r', encoding='utf-8') as f :
    text = f.read().splitlines()

#2. 불용어 리스트 불러오기

In [5]:
with open('/content/drive/MyDrive/AI_Study/NLP/DATA/전처리용 DATA/custom_stopwords.txt', 'r', encoding='utf-8') as f :
    stopwords = f.read().splitlines()  

#3. 명사 형태소로 구분후 불용어 제거 클래스

In [11]:
class Mecab_sentence(object):

    def __init__(self): 
        self.stopwords = stopwords
        self.mecab = Mecab()

    def nouns(self, sentences):
        nouns = []
        for sen in sentences:
            if sen != ' ' :
                nouns.append(" ".join([noun for noun in self.mecab.nouns(str(sen)) 
                                  if noun not in self.stopwords and len(noun) > 1]))
        return nouns

#4.TF-IDF 행렬 그래프와 빈도수 벡터 행렬 그래프 작성 클래스

In [12]:
class G_matrix(object):

    def __init__(self):
        self.TF_IDF = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
  
    def sent_matrix(self, nouns):
        TF_IDF_MAT = self.TF_IDF.fit_transform(nouns).toarray()
        graph_sentence = np.dot(TF_IDF_MAT, TF_IDF_MAT.T)    # 용어 빈도와 역 문서 빈도와의 통계 곱
        return graph_sentence

    def word_matrix(self, nouns):
        CNT_VEC_MAT = normalize(self.cnt_vec.fit_transform(nouns).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(CNT_VEC_MAT.T, CNT_VEC_MAT), {vocab[word] : word for word in vocab}

#5.TF-IDF 행렬 그래프와 연립 일차방정식을 이용하여 랭크를 태그

In [13]:
class Rank(object):

    def get_ranks(self, graph, d=0.85):
        A = graph
        matrix_size = A.shape[0]           # sent_graph 의 행 길이만큼(706)   
        for idx in range(matrix_size):     # sent_graph 의 행 길이만큼의 인덱스(0 ~ 705) 
            A[idx, idx] = 0                  # sent_graph 의 왼쪽위 기준으로 대각선을 0으로 변경(706 x 706)
            link_sum = np.sum(A[:, idx])     # sent_graph 의 열 마다 모두 더한다
            if link_sum != 0 :               # sent_graph 의 열 마다 모두 더한 값이 0이 아닌 경우에
                A[:, idx] /= link_sum
            A[:, idx] *= -d
            A[idx, idx] = 1

        B = (1-d) * np.ones((matrix_size, 1)) 
        x = np.linalg.solve(A, B)          # 연립방정식 Ax = B
        return {idx : r[0] for idx, r in enumerate(x)}

#6.키워드 or 문장 출력

In [14]:
class TextRank(object):

    def __init__(self, text):
     
        self.sentence = text
        self.sent_tokenize = Mecab_sentence()
        self.nouns = self.sent_tokenize.nouns(self.sentence)

        self.matrix = G_matrix()
        self.sent_graph = self.matrix.sent_matrix(self.nouns)
        self.word_graph, self.idx2word =  self.matrix.word_matrix(self.nouns)

        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)

        self.word_rank_idx = self.rank.get_ranks(self.word_graph)
        self.word_sent_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

    def key_sentence(self, sentence_num=5):
        summary = []
        index = []
        for idx in self.sorted_sent_rank_idx[:sentence_num]:
            index.append(idx)

        index.sort()
        for idx in index:
            summary.append(self.sentence[idx])

    
        return [w for w in summary]

    def key_words(self, word_num=10):
        rank_idx = self.rank.get_ranks(self.word_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
    
        keywords = []
        index=[]

        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
    
        return keywords

In [23]:
textrank = TextRank(text)

for i in textrank.key_sentence() : 


    print(f'핵심 문장 : {i}', '\n')
print('-' * 100, '\n')
print('핵심 키워드 :', textrank.key_words())

핵심 문장 : 그리곤 첫째 돼지와 둘째 돼지는 신나게 놀았어요. 

핵심 문장 : 늑대가 첫째 돼지의 집으로 갔어요. 

핵심 문장 : 첫째 돼지와 둘째 돼지는 셋째 돼지네 집으로 달려갔어요. 

핵심 문장 : 이윽고 뒤따라온 늑대가 셋째 돼지의 집을 보았어요. 

핵심 문장 : 그 후로 첫째 돼지와 둘째 돼지도 셋째 돼지의 집 근처에다 튼튼한 벽돌집을 지었대요. 

---------------------------------------------------------------------------------------------------- 

핵심 키워드 : ['돼지', '아기', '첫째', '셋째', '형제', '벽돌', '둘째', '늑대', '걱정', '꼴찌']
